In [2]:
print("\nRunning Update HOTzone Python Script\n\n\n")
print("Importing geopandas site package...\n")
import geopandas as gpd
import sys
import os
import imp
import time
import traceback

TAZ_shp = r'inputs-04-14\\TAZ.shp'
Scenario_Link = r'inputs-04-14\\C1_Link.shp'
Scenario_Node = r'inputs-04-14\\C1_Node.shp'
UsedZones = '3629'
temp_folder = r'temp_gpd'
log = os.path.join(temp_folder,r'LogFile_UpdateLinkNodeTAZID.txt')

# Print key variabls to screen
print("TAZ_shp: \n    "           + TAZ_shp           + "\n")
print("Scenario_Link: \n    "     + Scenario_Link     + "\n")
print("Scenario_Node: \n    "     + Scenario_Node     + "\n")
print("UsedZones: \n    "         + str(UsedZones)    + "\n")
print("Temp Folder:  \n    "      + temp_folder       + "\n")

# Open log file
logFile = open(log, 'w')
logFile.write(TAZ_shp+'\n')
logFile.write(Scenario_Link+'\n')
logFile.write(Scenario_Node+'\n')
logFile.write(str(UsedZones)+'\n')
logFile.write(temp_folder+'\n')

#raw_input()


## Define Variables
# Variables: Input
taz_shp           = str(TAZ_shp)
link_shp          = str(Scenario_Link)
node_shp          = str(Scenario_Node)
UsedZones         = str(UsedZones)
temp_folder       = str(temp_folder)

# Variables: Intermediate
link_taz_sj       = os.path.join(temp_folder, "Link_TAZ_SJ_deleteTemp.shp")
node_taz_sj       = os.path.join(temp_folder, "Node_TAZ_SJ_deleteTemp.shp")

# Variables: Output
out_link          = os.path.join(temp_folder, "C1_Link_TAZID.shp")
out_node          = os.path.join(temp_folder, "C1_Node_TAZID.shp")
out_link_mp       = os.path.join(temp_folder, "C1_Link_Midponts.shp")

# Codeblock to calculate TAZID
def calcTAZID_Node(tazid, node, global_n):
  if node <= int(global_n):
    return node
  else:
    return tazid

def calcTAZID_Link(tazid, aField, bField, global_n):
    if int(aField) <= int(global_n):
        return aField
    elif int(bField) <= int(global_n):
        return bField
    else:
        return tazid


Running Update HOTzone Python Script



Importing geopandas site package...

TAZ_shp: 
    inputs-04-14\\TAZ.shp

Scenario_Link: 
    inputs-04-14\\C1_Link.shp

Scenario_Node: 
    inputs-04-14\\C1_Node.shp

UsedZones: 
    3629

Temp Folder:  
    temp_gpd



c:\Users\cday\Anaconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\cday\AppData\Local\Temp\ipykernel_7548\2175388383.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

## TagLinksWithTAZID()

In [4]:
print("\n\nImporting Highway Link data...")
gdf_link = gpd.read_file(link_shp)




Importing Highway Link data...


In [5]:
print("\nCalculating Highway Link distance (in miles) and updating DISTANCE field...")
gdf_link["DISTANCE"] = gdf_link.geometry.length / 1609.34  # Conversion from meters to miles  (DO I NEED THIS CONVERSION?)


Calculating Highway Link distance (in miles) and updating DISTANCE field...


In [6]:
print("\nCalculating Highway Link midpoint coord and updating X_MID & Y_MID fields...")
gdf_link["X_MID"] = gdf_link.geometry.interpolate(0.5, normalized=True).x
gdf_link["Y_MID"] = gdf_link.geometry.interpolate(0.5, normalized=True).y


Calculating Highway Link midpoint coord and updating X_MID & Y_MID fields...


In [7]:
print("\nMaking new point shapefile from Highway Link midpoint...")
gdf_midpoints = gdf_link.copy()
gdf_midpoints = gdf_midpoints.set_geometry(gdf_midpoints.geometry.centroid)
gdf_midpoints.to_file(out_link_mp)
gdf_midpoints


Making new point shapefile from Highway Link midpoint...


,A,B,DISTANCE,DISTEXCEPT,EXTERNAL,LINKID,FLG_NEWNET,MAG_LINK,STREET,HOTZN,SEL_LINK,ONEWAY,DIRECTION,SFAC_BASE,SFAC_FUT,CFAC_BASE,CFAC_FUT,TRKRST2015,TRKRST2023,TAZID,SEGID,SEGEX_RTP,SEGEX_NEED,LN_2015,LN_2019,LN_2021,LN_2023,LN_2024,LN_2028,LN23_32,LN23_42,LN23_50,LN23_32UF,LN23_42UF,LN23_50UF,FT_2015,FT_2019,FT_2021,FT_2023,FT_2024,FT_2028,FT23_32,FT23_42,FT23_50,FT23_32UF,FT23_42UF,FT23_50UF,TSPD_2015,TSPD_2019,TSPD_2021,TSPD_2023,TSPD_2024,TSPD_2028,TSPD23_32,TSPD23_42,TSPD23_50,TSPD23_32U,TSPD23_42U,TSPD23_50U,TSPD_AVE,TRNSPD_PTC,TRNSPD_MIS,TRNSPD_MED,TRNSPD_HIG,TRNSPD_HII,HOT_2015,HOT_2019,HOT_2021,HOT_2023,HOT_2024,HOT_2028,HOT23_32,HOT23_42,HOT23_50,HOT23_32UF,HOT23_42UF,HOT23_50UF,REL_2015,REL_2019,REL_2021,REL_2023,REL_2024,REL_2028,REL23_32,REL23_42,REL23_50,REL23_32UF,REL23_42UF,REL23_50UF,OP_2019,OP_2021,OP_2023,OP_2024,OP_2028,OP23_32,OP23_42,OP23_50,OP23_32UF,OP23_42UF,OP23_50UF,GIS23_32,GIS23_42,GIS23_50,SCRN_LRG,SCRN_SML,FC_ID,SA_ID,LN19_30,LN19_40,LN19_50,LN19_30UF,LN19_40UF,LN19_50UF,LN19_50NNR,FT19_30,FT19_40,FT19_50,FT19_30UF,FT19_40UF,FT19_50UF,FT19_50NNR,TSPD19_30,TSPD19_40,TSPD19_50,TSPD19_30U,TSPD19_40U,TSPD19_50U,HOT19_30,HOT19_40,HOT19_50,REL19_30,REL19_40,REL19_50,OP19_30,OP19_40,OP19_50,OP19_30UF,OP19_40UF,OP19_50UF,OP19_50NNR,GIS19_30,GIS19_40,GIS19_50,SORT_L,geometry,X_MID,Y_MID
0,1,29685,0.387704,0,0,1_29685,0,0,Cent,0,0,0,EB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,POINT (412206.563 4605795.562),412206.562800,4.605796e+06
1,2,29685,0.108710,0,0,2_29685,0,0,Cent,0,0,0,WB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,POINT (412559.563 4605981.062),412559.562800,4.605981e+06
2,3,29645,0.518017,0,0,3_29645,0,0,Cent,0,0,0,EB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,POINT (409312.708 4603276.537),409312.707700,4.603277e+06
3,4,29671,0.254196,0,0,4_29671,0,0,Cent,0,0,0,WB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,POINT (409846.556 4604799.603),409846.555850,4.604800e+06
4,5,29680,0.294677,0,0,5_29680,0,0,Cent,0,0,0,NB,0.0,0.0,0.0,0.0,0,0,0,None,None,None,7,7,7,7,7,7,7,7,7,7,7,7,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,7,7,7,7,7,7,7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,POINT (413438.000 4603833.000),413438.000000,4.603833e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56472,95064,95025,1.515087,0,0,95064_95025,23,1,Sixth Water Creek,0,0,2,WB,0.0,0.0,0.0,0.0,0,0,0,None,None,Non

In [8]:
from shapely.geometry import box

def find_nearest_polygon(point_gdf, polygon_gdf,buffer_distance):
    point_gdf2 = point_gdf.copy()
    # Create a spatial index for the polygon geometries
    polygon_sindex = polygon_gdf.sindex
    
    nearest_tazid = []
    for point in point_gdf2.geometry:
        # Get the indices of candidate polygons based on the point's bounding box
        point_buffer = point.buffer(buffer_distance)
        expanded_bounds = box(*point_buffer.bounds)
        potential_nearest_idx = list(polygon_sindex.intersection(expanded_bounds.bounds))
        
        if len(potential_nearest_idx) > 0:
            # Calculate distances only for the potential candidate polygons
            distances = polygon_gdf.geometry.iloc[potential_nearest_idx].distance(point)
            
            # Find the index of the polygon with the minimum distance
            nearest_idx = distances.idxmin()
            
            # Retrieve the corresponding 'TAZID' value
            nearest_tazid.append(polygon_gdf.loc[nearest_idx, 'TAZID'])
        else:
            # Handle the case when there are no candidate polygons
            nearest_tazid.append(None)
            print('No TAZ polygon close enough to this link midpoint. Increase buffer size in code.')
    
    point_gdf2['nearest_tazid'] = nearest_tazid
    return point_gdf2


In [9]:
# read in taz shapefile and calculate which tazes are closest to each midpoint value
gdf_taz = gpd.read_file(taz_shp)
gdf_midpoints_nearest_taz = find_nearest_polygon(gdf_midpoints,gdf_taz,1000)
gdf_midpoints_nearest_taz = gdf_midpoints_nearest_taz[['LINKID','nearest_tazid']]

In [10]:
drop_columns = ['TAZID_V832', 'SORT', 'CO_IDX', 'CO_TAZID', 'SUBAREAID', 'ACRES', 'DEVACRES', 'DEVPBLEPCT', 'X', 'Y', 'ADJ_XY', 'CO_FIPS', 'CO_NAME', 'CITY_FIPS', 'CITY_UGRC', 'CITY_NAME', 'DISTSUPER', 'DSUP_NAME', 'DISTLRG', 'DLRG_NAME', 'DISTMED', 'DMED_NAME', 'DISTSML', 'DSML_NAME', 'CBD', 'TERMTIME', 'PRKCSTPERM', 'PRKCSTTEMP', 'WALK100', 'ECOEDPASS', 'FREEFARE', 'REMM']

In [11]:
import numpy as np
print("\nSpatial joining TAZ to Link midpoints (this may take a few minutes)...")
# get the tazid for all those midpoints that exist within a tazid
gdf_link_taz_sj = gpd.sjoin(gdf_midpoints, gdf_taz, how="left", op="within")
gdf_link_taz_sj = gdf_link_taz_sj.rename(columns={'TAZID_left':'TAZID','TAZID_right':'TAZID_1'})
gdf_link_taz_sj = gdf_link_taz_sj.drop(columns=drop_columns)

# update TAZID_1 to be the value of the nearest tazid to the midpoint for those midpoints that don't exist within a taz polygon
gdf_link_taz_sj_F = gdf_link_taz_sj.merge(gdf_midpoints_nearest_taz, on = 'LINKID', how = 'left')
gdf_link_taz_sj_F['TAZID_1'] = np.where(gdf_link_taz_sj_F['TAZID_1'].isna(), gdf_link_taz_sj_F['nearest_tazid'], gdf_link_taz_sj_F['TAZID_1'])

# get missing column data from taz dataframe
df_taz = pd.DataFrame(gdf_taz).drop(columns={'geometry'})
gdf_link_taz_sj_F = gdf_link_taz_sj_F.merge(df_taz, left_on='TAZID_1',right_on='TAZID', how='left')
gdf_link_taz_sj_F = gdf_link_taz_sj_F.rename(columns={'TAZID_x':'TAZID'}).drop(columns={'TAZID_y'})
gdf_link_taz_sj_F.to_file(link_taz_sj)


Spatial joining TAZ to Link midpoints (this may take a few minutes)...


c:\Users\cday\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\cday\AppData\Local\Temp\ipykernel_7548\1237057531.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_link_taz_sj_F.to_file(link_taz_sj)


In [12]:
#neartest = gpd.sjoin_nearest(gdf_midpoints, gdf_taz, how = 'left')

In [13]:
print("\nUpdating Highway Link TAZID...\n")
gdf_link_mp = gdf_link_taz_sj_F.copy()
gdf_link_mp["TAZID"] = gdf_link_mp.apply(lambda row: calcTAZID_Link(row["TAZID_1"], row["A"], row["B"], UsedZones), axis=1)
gdf_link_mp = gdf_link_mp.drop(columns=drop_columns).drop(columns={'nearest_tazid','TAZID_1'})
gdf_link_mp.to_file(out_link)


Updating Highway Link TAZID...



C:\Users\cday\AppData\Local\Temp\ipykernel_7548\4204474982.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_link_mp.to_file(out_link)


# TagNodesWithTAZID()

In [14]:
print("\n\nImporting Highway Node data...")
gdf_nodes = gpd.read_file(node_shp)



Importing Highway Node data...


,N,X,Y,GEOGKEY,EXTERNAL,FLG_NEWNET,MAG_NODE,UNCON,HOTZN,TAZID,NODENAME,PNR_2015,PNR_2019,PNR_2021,PNR_2023,PNR_2024,PNR_2028,PNR23_32,PNR23_42,PNR23_50,PNR23_32UF,PNR23_42UF,PNR23_50UF,FARZN_2015,FARZN_2019,FARZN_2021,FARZN_2023,FARZN_2024,FARZN_2028,FARZN23_32,FARZN23_42,FARZN23_50,FARE23_32U,FARE23_42U,FARE23_50U,PNR19_30,PNR19_40,PNR19_50,FARZN19_30,FARZN19_40,FARZN19_50,SORT_N,geometry
0,1,411935.0000,4605642.000,411935_4605642,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,POINT (411935.000 4605642.000)
1,2,412641.0000,4606013.000,412641_4606013,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,POINT (412641.000 4606013.000)
2,3,408993.0000,4603544.000,408993_4603544,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,POINT (408993.000 4603544.000)
3,4,410051.0000,4604806.000,410051_4604806,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,POINT (410051.000 4604806.000)
4,5,413394.0000,4603600.000,413394_4603600,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,POINT (413394.000 4603600.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21216,95061,431050.6881,4422110.715,431050_4422110,0,23,1,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22081,POINT (431050.688 4422110.715)
21217,95062,430713.6121,4421517.072,430713_4421517,0,23,1,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22082,POINT (430713.612 4421517.072)
21218,95063,452850.1772,4468529.559,452850_4468529,0,23,1,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22083,POINT (452850.177 4468529.559)
21219,95064,473181.5500,4444171.200,473181_4444171,0,23,1,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22084,POINT (473181.550 4444171.200)


In [15]:
print("\nSpatial joining TAZ to Highway Nodes (this may take a few minutes)...")
gdf_taz = gpd.read_file(taz_shp)
gdf_nodes_nearest_taz = find_nearest_polygon(gdf_nodes,gdf_taz,1000)

# convert taz geodataframe to regular dataframe
df_taz = pd.DataFrame(gdf_taz).drop(columns={'geometry'}).rename(columns={'X':'X_1','Y':'Y_1', 'TAZID':'TAZID_1'})
gdf_node_taz_sj = gdf_nodes_nearest_taz.merge(df_taz, left_on='nearest_tazid',right_on='TAZID_1', how='left')
gdf_node_taz_sj = gdf_node_taz_sj.drop(columns={'nearest_tazid'})
gdf_node_taz_sj.to_file(node_taz_sj)


Spatial joining TAZ to Highway Nodes (this may take a few minutes)...


In [16]:
print("\nUpdating Highway Node TAZID...\n")
gdf_node_mp = gdf_nodes_nearest_taz.copy()
gdf_node_mp['TAZID'] = gdf_node_mp.apply(lambda row: calcTAZID_Node(row['nearest_tazid'], row['N'], UsedZones), axis = 1)
gdf_node_mp = gdf_node_mp.drop(columns={'nearest_tazid'})
gdf_node_mp.to_file(out_node)


Updating Highway Node TAZID...

